## Introductie

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

Alle code die specifiek is gemaakt voor deze modelbouw is geschreven in het Nederlands. Code die specifiek relateert aan D-HYDAMO of D-HYDRO is geschreven in het Engels.

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.

In [1]:
from pathlib import Path

beheerregister_pad = Path(r".\beheerregister").absolute().resolve()

invoerbestanden = {"modelgebied": "Bemalings_en_lozingsgebieden.shp",
                   "branches": "Afaanvoervakken_DHYDRO_v3.shp",
                   "profielpunten": "profielpunten_alles_10dec2020.shp",
                   "bruggen": "Bruggen_(doorstroomopeningen).shp",
                   "duikers": "duikers.shp",
                   "sifons": "sifons.shp",
                   "stuwen": "stuwen.shp",
                   "inlaten": "inlaten.shp",
                   "gemalen": "gemalen.shp",
                   "peilgebieden": "Peilgebieden.shp"}

for key, item in invoerbestanden.items():
    if not beheerregister_pad.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister_pad.joinpath(item)}")

## Aanmaken HYDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hkvsobekpy
import hydrotools
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import pandas as pd
from shapely.geometry import Polygon, LineString, Point
from shapely.ops import snap

hydamo = HyDAMO(extent_file=str(beheerregister_pad.joinpath(invoerbestanden["modelgebied"])))

## Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [3]:
gdf = hydrotools.read_file(beheerregister_pad.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"HydroModel": "Boezemmodel_v4"},
                           column_mapping={"OVKIDENT": "code"},
                           )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 4

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

## Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [5]:
gdf = hydrotools.read_file(beheerregister_pad.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["WL", "Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])